Parte A

Levantar cluster en AWS con Hadoop y Pyspark
Nombre: cluster_ + la mátricula (número de estudiante) más chica

Acceso a JupyterHub
JupyterHub: https://master-public-dns-name:9443/
- u: `jovyan`
- p: `jupyter`

Librerías y sesión de Spark

In [4]:
# Crear sesión de SPARK y Cargar los datos 
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, to_date, countDistinct, col, max as max_, row_number, collect_list, array_contains
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

BUCKET = "s3://proyecto-parcial-equipo-6/" #VALIDAR CARPETA EXACTA CON LAURO

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------------------------+--------------------+---------------------+-----------------+-------+----------+------+---------------------------+--------------------------------------+------------------------------------------------------------------------------------------+----------------+----------------+---------+-----------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLOR BLANCA O NEGRA|               ALEXA|APARATOS ELECTRONICOS|ELECTRODOMESTICOS|2499.00|2023-05-02|COPPEL|TIENDA DE ELECTRODOMÉSTICOS|COPPEL SUCURSAL ESPACIO AGUASCALIENTES|TECNOLOGICO 120, INT. CENTRO COMERCIAL ESPACIO AGUASCALIENTES, COL. OJOCALIENTE, CP. 20000|AGUASCALIENTES11|AGUASCALIENTES12|21.879490|-102.260011|
+------------------+---------------------------------+--------------------+---------------------+-----------------+-------+----------+------+---------------------------+--------------------------------------+--------------------------------------------------------------------------------

Carga el CSV en Spark

In [ ]:
# Revisión de estructura de datos antes de tener los headers
# Inicia una sesión Spark.
spark = SparkSession.builder.appName("parteA").getOrCreate()

# Lee solo un archivo CSV de muestra desde S3.
df = spark.read.csv(BUCKET + "05-2023_01.csv", header=False, inferSchema=True, sep='|')

# Muestra las primeras filas para asegurarte de que se ha cargado correctamente.
df.show()

Carga el CSV en Spark
Adición de headers

In [22]:
# Renombrar las columnas del DataFrame.
df_with_headers = df.toDF("producto", "presentacion", "marca", "categoria", "catalogo", "precio", "fecha_registro", "cadena_comercial", "giro", "nombre_comercial", "direccion", "estado", "municipio", "latitud", "longitud")

# Muestra las primeras filas para asegurarte de que se han cargado correctamente con los nuevos encabezados.
df_with_headers.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+-----------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
|          producto|        presentacion|               marca|           categoria|        catalogom| precio|fecha_registro|cadena_comercial|                giro|    nombre_comercial|           direccion|        estado|     municipio|  latitud|   longitud|
+------------------+--------------------+--------------------+--------------------+-----------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLO...|               ALEXA|APARATOS ELECTRON...|ELECTRODOMESTICOS| 2499.0|    2023-05-02|          COPPEL|TIENDA DE ELECTRO...|COPPEL SUCURSAL E...|TECNOLOGICO 120, ...|AGUASCALIENTES|AGUASCALIE

Guarda el CSV como parquet en S3, particionalo por catalogo. (Utiliza todos los trucos que consideres).

In [9]:
# Guarda el CSV como parquet en S3, particionalo por catalogo
df_with_headers.write.partitionBy("catalogo").parquet(BUCKET + "output/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carga el parquet en Spark

In [10]:
# Carga del parquet en Spark
parquet_df = spark.read.parquet(BUCKET + "output/")

# Muestra las primeras filas para verificar.
parquet_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+-----------------+
|          producto|        presentacion|               marca|           categoria| precio|fecha_registro|cadena_comercial|                giro|    nombre_comercial|           direccion|        estado|     municipio|  latitud|   longitud|        catalogom|
+------------------+--------------------+--------------------+--------------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+-----------------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLO...|               ALEXA|APARATOS ELECTRON...| 2499.0|    2023-05-02|          COPPEL|TIENDA DE ELECTRO...|COPPEL SUCURSAL E...|TECNOLOGICO 120, ...|AGUASCALIENTES|AGUASCALIENTES|21.879490|-10

¿Cuántos catálogos diferentes tenemos?

In [12]:
catalog_count_by_year = (parquet_df
    .withColumn("fecha_registro", to_date("fecha_registro", "yyyy-MM-dd"))  # Convierte la cadena de fecha en tipo de dato Date
    .groupBy(year("fecha_registro").alias("año"))  # Extrae el año y agrupa por este
    .agg(countDistinct("catalogom").alias("distinct_catalogs"))  # Cuenta los catálogos únicos
)

# Muestra el resultado.
catalog_count_by_year.show()

# Guarda la salida en el bucket de S3.
catalog_count_by_year.write.mode("overwrite").parquet(BUCKET + "preguntas/diff_catalogs/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------+
| a?o|distinct_catalogs|
+----+-----------------+
|2023|                7|
+----+-----------------+

¿Cuáles son los 20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.

In [14]:
# Encuentra los 20 catálogos con más observaciones.
top_catalogs = parquet_df.groupBy("catalogom").count().orderBy(col("count").desc()).limit(20)

# Muestra el resultado.
top_catalogs.show()

# Guarda la salida en el bucket de S3.
top_catalogs.write.mode("overwrite").parquet(BUCKET + "preguntas/top_catalogs/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+
|          catalogom| count|
+-------------------+------+
|            BASICOS|283314|
|       MEDICAMENTOS|112418|
|  ELECTRODOMESTICOS| 37306|
| FRUTAS Y LEGUMBRES| 32185|
|              PACIC| 26481|
|           MERCADOS| 11745|
|PESCADOS Y MARISCOS|  2282|
+-------------------+------+

¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

In [16]:
# Lista todos los estados únicos presentes en el DataFrame.
unique_states = parquet_df.select("estado").distinct()

# Muestra los estados únicos.
unique_states.show()

# Opcional: Guarda la salida en el bucket de S3.
unique_states.write.mode("overwrite").csv(BUCKET + "preguntas/unique_states/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|              estado|
+--------------------+
|             YUCATÁN|
|            VERACRUZ|
|              OAXACA|
|              PUEBLA|
|        QUINTANA ROO|
|             TABASCO|
|             SINALOA|
|             CHIAPAS|
|              SONORA|
|             DURANGO|
|            TLAXCALA|
|COAHUILA DE ZARAGOZA|
|           CHIHUAHUA|
|     SAN LUIS POTOSÍ|
|          TAMAULIPAS|
| BAJA CALIFORNIA SUR|
|             JALISCO|
|          NUEVO LEÓN|
|    ESTADO DE MÉXICO|
|           ZACATECAS|
+--------------------+
only showing top 20 rows

In [21]:
# all_states es una lista de todos los estados posibles del país.
all_states = ["AGUASCALIENTES", "BAJA CALIFORNIA", "BAJA CALIFORNIA SUR", "CAMPECHE", "CHIAPAS","CHIHUAHUA","CIUDAD DE MEXICO",
"COAHUILA DE ZARAGOZA","COLIMA","DURANGO","ESTADO DE MEXICO","GUANAJUATO","GUERRERO","HIDALGO","JALISCO","MICHOACÁN","MORELOS","NAYARIT",
"NUEVO LEON","OAXACA","PUEBLA","QUERETARO","QUINTANA ROO","SAN LUIS POTOSI","SINALOA","SONORA","TABASCO","TAMAULIPAS","TLAXCALA",
"VERACRUZ","YUCATAN","ZACATECAS"]

# Crea un DataFrame para la lista completa de estados.
all_states_df = spark.createDataFrame([(state,) for state in all_states], ["estado"])

# Realiza un anti join para encontrar los estados que faltan.
missing_states = all_states_df.join(unique_states, "estado", "left_anti")

# Muestra los estados que faltan.
missing_states.show()

# Opcional: Guarda la salida en el bucket de S3.
missing_states.write.mode("overwrite").csv(BUCKET + "preguntas/missing_states/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|          estado|
+----------------+
|CIUDAD DE M?XICO|
|          COLIMA|
|ESTADO DE M?XICO|
|       MICHOAC?N|
|         NAYARIT|
|      NUEVO LE?N|
|       QUER?TARO|
| SAN LUIS POTOS?|
|         YUCAT?N|
+----------------+

¿Cuántas observaciones tenemos por estado?

In [28]:
# Agrupa por estado y cuenta las observaciones
observations_per_state = parquet_df.groupBy("estado").count().orderBy(col("count").desc())

# Muestra los resultados
observations_per_state.show()

# Opcional: Guarda la salida en el bucket de S3.
observations_per_state.write.mode("overwrite").csv(BUCKET + "preguntas/observations_per_state/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+
|             estado| count|
+-------------------+------+
|   CIUDAD DE MÉXICO|117779|
|   ESTADO DE MÉXICO| 80309|
|            JALISCO| 24028|
|         GUANAJUATO| 23527|
|         NUEVO LEÓN| 17754|
|            TABASCO| 17120|
|             PUEBLA| 16857|
|          QUERÉTARO| 16517|
|          CHIHUAHUA| 15750|
|            YUCATÁN| 14231|
|          ZACATECAS| 13775|
|       QUINTANA ROO| 12486|
|           CAMPECHE| 11650|
|BAJA CALIFORNIA SUR| 10338|
|     AGUASCALIENTES|  9957|
|            DURANGO|  9878|
|           TLAXCALA|  9766|
|MICHOACÁN DE OCAMPO|  9703|
|             SONORA|  9055|
|             OAXACA|  8813|
+-------------------+------+
only showing top 20 rows

De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [38]:
# Define una ventana particionada por estado y ordenada por año para análisis temporal.
windowSpec = Window.partitionBy("estado").orderBy("year")

# Preparar el DataFrame y calcular el número de catálogos distintos por estado y año, y el cambio año a año.
catalogs_per_state_year = (
    parquet_df
    # Asegurarse de que 'fecha_registro' está en formato de fecha correcto.
    .withColumn("fecha_registro", to_date("fecha_registro", "yyyy-MM-dd"))
    # Extraer el año y agrupar por este y por estado.
    .groupBy("estado", year("fecha_registro").alias("year"))
    .agg(countDistinct("catalogom").alias("num_catalogs"))
    # Calcular el número de catálogos del año anterior y el cambio respecto al año anterior.
    .withColumn("prev_year_catalogs", lag("num_catalogs", 1).over(windowSpec))
    .withColumn("change", col("num_catalogs") - col("prev_year_catalogs"))
    .select("estado", "year", "num_catalogs", "change")
    .orderBy("estado", "year")
)

# Muestra los resultados
catalogs_per_state_year.show()

# Opcional: Guarda la salida en el bucket de S3.
catalogs_per_state_year.write.mode("overwrite").csv(BUCKET + "preguntas/catalogs_per_state_year/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+------------+------+
|              estado|year|num_catalogs|change|
+--------------------+----+------------+------+
|      AGUASCALIENTES|2023|           7|  NULL|
|     BAJA CALIFORNIA|2023|           7|  NULL|
| BAJA CALIFORNIA SUR|2023|           7|  NULL|
|            CAMPECHE|2023|           7|  NULL|
|             CHIAPAS|2023|           7|  NULL|
|           CHIHUAHUA|2023|           7|  NULL|
|    CIUDAD DE MÉXICO|2023|           7|  NULL|
|COAHUILA DE ZARAGOZA|2023|           7|  NULL|
|             DURANGO|2023|           7|  NULL|
|    ESTADO DE MÉXICO|2023|           7|  NULL|
|          GUANAJUATO|2023|           7|  NULL|
|            GUERRERO|2023|           3|  NULL|
|             HIDALGO|2023|           7|  NULL|
|             JALISCO|2023|           7|  NULL|
| MICHOACÁN DE OCAMPO|2023|           6|  NULL|
|             MORELOS|2023|           7|  NULL|
|          NUEVO LEÓN|2023|           7|  NULL|
|              OAXACA|2023|           5|

Utilizando Spark contesta las siguientes preguntas a partir del catálogo que le tocó a tu equipo. Recuerda trabajar en el archivo con los datos particionados de otra manera tus queries van a tardar mucho.

¿Cuańtas marcas diferentes tiene tu categoría?

In [ ]:
# Filtra por catalogo asignado
filtered_df = parquet_df.filter(col("catalogo") == "BASICOS")

# Selecciona la columna 'marca', encuentra valores únicos, y cuenta
unique_brands_count = filtered_df.select("marca").distinct().count()

print(f"Numero de marcas unicas en la catalogo: 'Basicos': {unique_brands_count}")

¿Cuál es la marca con mayor precio? ¿En qué estado?

In [ ]:
# Encuentra la marca con el precio más alto y el estado correspondiente
max_price_info = filtered_df.orderBy(col("precio").desc()).select("marca", "precio", "estado").first()

print(f"La marca con el precio más alto es {max_price_info['marca']} con un precio de {max_price_info['precio']} en el estado de {max_price_info['estado']}.")

¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [ ]:
# Filtra por estados que contengan la secuencia 'CIUDAD DE'
df_cdmx = filtered_df.filter(col("estado").contains("CIUDAD DE"))

# Encuentra la marca con el precio más bajo y el estado correspondiente
min_price_info = df_cdmx.orderBy(col("precio").asc()).select("marca", "precio", "municipio").first()

print(f"La marca con el precio más bajo en Ciudad de México es {min_price_info['marca']} con un precio de {min_price_info['precio']}.")

¿Cuál es la marca con mayores observaciones?

In [ ]:
# Agrupa por la columna 'marca' y cuenta el número de observaciones para cada marca
brand_counts = filtered_df.groupBy("marca").count()

# Ordena los resultados por el conteo en orden descendente y toma la primera fila
most_observed_brand = brand_counts.orderBy(col("count").desc()).first()

# Verifica si hay resultados antes de acceder al resultado
if most_observed_brand:
    print(f"La marca con el mayor número de observaciones es {most_observed_brand['marca']}, con {most_observed_brand['count']} observaciones.")
else:
    print("No se encontraron registros en el dataset.")

¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

In [ ]:
# Definir una ventana ordenada por precio descendente dentro de cada estado
windowSpec = Window.partitionBy("estado").orderBy(col("precio").desc())

# Aplicar la ventana con un número de fila para identificar los 5 principales
state_brand_prices = filtered_df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter(col("row_number") <= 5) \
    .select("estado", "marca", "precio", "row_number")

state_brand_prices.show()

# Para comparar las 5 listas principales en diferentes estados, las recopilamos e imprimimos agrupadas por estado.
top_brands_by_state = state_brand_prices.groupBy("estado") \
    .agg(collect_list("marca").alias("top_5_brands")) \
    .orderBy("estado")
top_brands_by_state.show(50, False)


¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [ ]:
# Define una ventana ordenada por precio ascendente dentro de cada estado que contiene "CIUDAD DE".
windowSpec = Window.partitionBy("estado").orderBy(col("precio").asc())

# Aplica la especificación de ventana con un número de fila para identificar el top 5 de precios más bajos.
lowest_price_brands = filtered_df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter(col("row_number") <= 5) \
    .select("estado", "marca", "precio", "row_number")

# Muestra las 5 marcas con los precios más bajos en los estados que contienen "CIUDAD DE".
lowest_price_brands.show()

# Para comparar las listas del top 5 entre diferentes "CIUDAD DE", las recopilamos y las mostramos agrupadas por estado.
top_brands_by_state_lowest = lowest_price_brands.groupBy("estado") \
    .agg(collect_list("marca").alias("top_5_lowest_brands")) \
    .orderBy("estado")
top_brands_by_state_lowest.show(50, False)

¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?

In [ ]:
# Contar las ocurrencias de cada marca en todo el dataset
brand_occurrences = filtered_df.groupBy("marca").count()

# Ordenar las marcas por el número de observaciones en orden descendente y seleccionar el top 5
top_5_brands_observations = brand_occurrences.orderBy(col("count").desc()).limit(5)

# Mostrar el top 5 de marcas con mayores observaciones
top_5_brands_observations.show()

In [ ]:
# Buscar las marcas top 5 de observaciones en una lista
top_5_observation_list = [row['marca'] for row in top_5_brands_observations.collect()]

# Ahora creamos el DataFrame especificando el esquema
top_5_obs_df = spark.createDataFrame(top_5_observation_list, StringType()).withColumnRenamed("value", "marca")
top_5_obs_df.show()

# Realizar un join entre el DataFrame de top 5 por precio y el DataFrame de observaciones top 5
comparison_df = state_brand_prices.join(top_5_obs_df, "marca", "inner")

# Mostrar los resultados de la comparación
comparison_df.show()

# Comprobar cuántos estados tienen marcas que están en las top 5 de observaciones
distinct_states_with_top5_obs = comparison_df.select("estado").distinct().count()
print(f"Número de estados que tienen al menos una marca en común con las top 5 de observaciones: {distinct_states_with_top5_obs}")

¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

In [ ]:
# Extraer el año de la columna 'registro' que tiene formato DD-MM-YYYY
# Convertir la columna de registro a tipo fecha y extraer el año
filtered_df = filtered_df.withColumn("fecha", to_date(col("fecha_registro"), "dd-MM-yyyy"))
filtered_df = filtered_df.withColumn("fecha_y", year(col("fecha")))

# Mostrar las primeras filas para verificar
filtered_df.show()

In [ ]:
# Ventana para particionar por marca
window_spec = Window.partitionBy("marca")

# Encontrar el último año de aparición para cada marca
last_appearance = filtered_df.withColumn("last_year", max_("fecha_y").over(window_spec))
last_appearance.show()

In [ ]:
# Seleccione columnas distintas de marca y último año para evitar filas duplicadas
distinct_last_appearance = last_appearance.select("marca", "last_year").distinct()
distinct_last_appearance.show()

Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.